<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/transformer/cifar10_pytorch_vit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Transformer (ViT) による画像分類

### はじめに

前回、CNN（畳み込みニューラルネットワーク）を使用してCIFAR-10データセットの分類を行いました。CNNでは、畳み込み層やプーリング層を使用して画像から特徴量を抽出し、最終的に分類を行います。しかし、今回の課題では畳み込み層やプーリング層を使わずに、同様のタスクを Vision Transformer (ViT) を使って実行してみましょう。

### ViTとは

<a title="Davide Coccomini, CC BY-SA 4.0 &lt;https://creativecommons.org/licenses/by-sa/4.0&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Vision_Transformer.gif"><img width="512" alt="Vision Transformer" src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Vision_Transformer.gif/512px-Vision_Transformer.gif?20230818142429"></a>

Vision Transformer (ViT) は、自然言語処理で成功を収めたTransformerモデルを画像認識に適用したものです。TransformerはAttentionメカニズムを使用して、入力データの重要な部分に焦点を当てることで特徴量を抽出します。これにより、ViTは画像全体を処理するのではなく、部分的な情報からも高い特徴抽出能力を発揮します。

### CNN と ViT

CNNの場合、以下の2種類の層を繰り返し用いて特徴量を抽出します。

- **畳み込み層**<br>
異なるフィルタを画像に適用して、エッジやテクスチャなどの低レベルの特徴を検出します。
- **プーリング層**<br>
空間的な次元を削減し、計算量を減らしながら重要な特徴を保持します。

CNN におけるこれらの処理では、**固定のカーネル** を使用し、局所的な特徴を抽出します。局所的なパターン認識に優れていますが、画像全体の文脈を直接捉えることは難しいです。
畳み込み層により、画像全体に同じフィルタが適用されます。

ViTは、特徴量のうち識別に有効な部分だけを重み付けします。自己注意機構を使用して、各パッチのうち、分類に有効そうなものに高い重みを与えるようになっています。これにより、もし

### 特徴抽出の流れ

ViT の場合は、以下の手順で特徴量を抽出します。

<img src="https://github.com/shizoda/education/assets/34496702/f4d8612f-66a6-4302-8163-dd5d8d63f27d" width=60%>

*[原論文 [Dosovitskiy21]](https://openreview.net/forum?id=YicbFdNTTy) Fig. 1 の一部。全体構成を表す*

- **パッチ分割**<br>
画像を小さな固定サイズのパッチに分割します。例えば、32x32のCIFAR-10の画像を4x4のパッチに分割すると、各パッチのサイズは8x8となり、合計64個のパッチが得られます。
- **パッチ埋め込み** <br>
各パッチを固定長のベクトルに変換します。各パッチをフラットにして線形層に通すことで行います。例えば、8x8のパッチは64次元のベクトルに変換され、それを指定した次元の埋め込みベクトル（例えば128次元）へ線形層により変換します。
- **位置エンコーディング** <br>
上で得られた埋め込みベクトルにはパッチの位置情報が与えられていないため、モデルは各パッチがどの位置にあるかを認識できません。位置エンコーディングは、固定長のベクトルを加算することで、各パッチに位置情報を持たせます。モデルが画像の構造を理解するのに役立ちます。
- **クラストークン**<br>
画像全体の特徴を表すために導入される特別なトークンです。このトークンは、全てのパッチ埋め込みと同じ次元を持ち、最初に入力として追加されます。最終的な分類結果は、このクラストークンの表現から得られます。
- **Transformerエンコーダ** <br>
パッチ埋め込みと位置エンコーディングを行った後、これらのベクトルはTransformerエンコーダに入力されます。エンコーダは複数の層からなり、各層は自己注意機構とフィードフォワードネットワークから構成されます。自己注意機構により、各パッチが他の全てのパッチとの関係を学習します。これにより、画像全体の文脈を考慮した特徴量を抽出できます。

### ViT のメリット

本来はもっと幅広いメリットがあるのですが、今回は Attention（注意）メカニズムに初めて触れる意味で、その名からわかりやすい以下の２点に着目してもらえれば十分と思います。

- **部分的な情報からの特徴抽出**<br>
ViTは、画像の一部にしか対象物が映っていない場合でも、その部分から有効な特徴量を抽出できます。これは自己注意機構 (Self-Attention Mechanism) が、画像中の重要な部分のパッチに大きな重みを与え、その部分からの特徴量を主として画像を表すからです。
- **自己注意機構による可視化**<br>
モデルがどの部分に注目するかを可視化することもできます。これにより、判断根拠をある程度可視化することが可能です。

### パッチ間の相互関係と画像理解

Transformerの自己注意機構では、各パッチが他の全てのパッチに対して注意を向けることができます。これにより、画像の一部分の情報が他の部分の情報とどのように関係しているかを学習できます。例えば、画像中のある物体の一部が他の部分とどのように連携しているかを理解することが可能です。これにより、画像全体のコンテキストを捉えることができ、より精度の高い特徴抽出と分類が可能になります。

---

In [ ]:
# PyTorch 関連のライブラリをインポートします
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import matplotlib.pyplot as plt
import numpy as np

# GPU が利用可能であることを確認
assert torch.cuda.is_available(), "GPU が使えません。ランタイムの設定を確認してください。"

### ViTクラスの定義

ViTクラスは、ViTモデル全体を定義するクラスです。画像をパッチに分割し、各パッチを埋め込んだ後、Transformerエンコーダを通じて特徴量を抽出し、最終的にクラスを予測します。

今回の実装では、以下の表のようにサイズ (shape) が変わっていきます。

| ステージ | 形状　　　　　　　　　　　　. |
|----------|-------------------|
| 入力画像 | $(B, 3, 32, 32)$ |
| パッチ分割 | $(B, 64, 48)$ |
| パッチ埋め込み | $(B, 64, 512)$ |
| クラストークン追加後 | $(B, 65, 512)$ |
| 位置エンコーディング追加後 | $(B, 65, 512)$ |
| Transformerエンコーダ後 | $(B, 65, 512)$ |
| 最終分類 (クラストークンの抽出) | $(B, 512)$ |
| 線形層通過後 | $(B, クラス数)$ |

#### 入力画像サイズ
- 入力画像のサイズは $(C, H, W)$ です。<br>
ここでは $C = 3$、$H = W = 32$ です。

#### パッチ分割
- パッチサイズを $P\times P$ とすると、ここでは $P=4$ です。
- したがって、パッチの数は
$$(H \times W)/(P \times P)=64$$
となります

#### パッチ埋め込み
- 各パッチの次元は $ 3 \times P^2 = 3 \times 4^2 = 48 $ です。
- パッチ埋め込み層では、これを埋め込み次元 $ D = 512 $ のベクトルに変換します。つまり、パッチの埋め込みベクトルの形状は$(B, 64, 512)$ になります。ここで $B$ はバッチサイズです。

#### クラストークンと位置エンコーディング
- **クラストークン（class token）** は、クラスごとに画像全体の特徴を表現するために使用されます。これは、パッチ埋め込みベクトルの先頭に追加され、形状は $ (B, 1, 512)$ となります。
- **位置エンコーディング（position embedding）** は、各パッチの位置情報を保持するために使用されます。これにより、モデルはパッチの順序を認識できます。位置エンコーディングを加えた後の入力ベクトルの形状は  $ (B, 65, 512)  $ となります。

#### Transformerエンコーダ
- **Transformerエンコーダ**は、自己注意機構を用いて各パッチ間の関係を学習します。入力の形状は $ (B, 65, 512) $ です。
- 各エンコーダ層では、入力ベクトルが自己注意機構とフィードフォワードネットワークを通過し、最終的に LayerNorm によって正規化されます。

#### 最終分類
- 最後に、クラストークンの表現が最終的な特徴ベクトルとして取り出され、線形層を通じてクラス分類が行われます。出力形状は $ (B, クラス数) $ です。

In [ ]:
# ViTクラスの定義
class ViT(nn.Module):
    def __init__(self, num_classes=10, img_size=32, patch_size=4, dim_hidden=512, num_heads=8, dim_feedforward=512, num_layers=6):
        super().__init__()
        assert img_size % patch_size == 0  # 画像サイズがパッチサイズで割り切れるか確認

        # 初期化パラメータの設定
        self.img_size = img_size
        self.patch_size = patch_size
        num_patches = (img_size // patch_size) ** 2  # パッチの数を計算
        dim_patch = 3 * patch_size ** 2  # 各パッチの次元数

        # パッチ埋め込み層
        self.patch_embed = nn.Linear(dim_patch, dim_hidden)  # パッチを埋め込みベクトルに変換
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, dim_hidden))  # 位置エンコーディングの初期化
        self.class_token = nn.Parameter(torch.zeros(1, 1, dim_hidden))  # クラストークンの初期化

        # Transformerエンコーダ層のスタック
        self.layers = nn.ModuleList([TransformerEncoderLayer(dim_hidden, num_heads, dim_feedforward) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(dim_hidden)  # 最後のLayerNorm層
        self.linear = nn.Linear(dim_hidden, num_classes)  # 最終分類用の線形層

    def forward(self, x):
        bs, c, h, w = x.shape  # バッチサイズ、チャンネル、高さ、幅
        assert h == self.img_size and w == self.img_size  # 入力画像サイズが一致するか確認

        # 画像をパッチに分割し、各パッチを埋め込みベクトルに変換
        x = x.view(bs, c, h // self.patch_size, self.patch_size, w // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 1, 3, 5).reshape(bs, -1, 3 * self.patch_size ** 2)
        x = self.patch_embed(x)

        # クラストークンを追加し、位置エンコーディングを適用
        class_token = self.class_token.expand(bs, -1, -1)
        x = torch.cat((class_token, x), dim=1)
        x += self.pos_embed

        # Transformerエンコーダ層を通過
        for layer in self.layers:
            x = layer(x)

        # attention_weightsを保存
        self.attention_weights = self.layers[0].attention.attn_weights

        # 最終分類
        x = self.norm(x)[:, 0]
        return self.linear(x)

#### **課題１**
全体的なネットワークの図を描いてください。
<br> 各ステップごとに shape がどうなるかも書き込んでください。

### TransformerEncoderLayer クラスの定義

TransformerEncoderLayer クラスは、1つのTransformerエンコーダ層を定義します。各エンコーダ層は自己注意機構とフィードフォワードネットワーク（FNN）から構成され、入力データの相互関係を学習します。

エンコーダ層の役割は、入力シーケンス内の異なる要素間の関係を学習することです。これにより、画像内の異なるパッチ間の関係を理解し、より豊かな特徴表現を得ることができます。

具体的には、エンコーダ層では以下の手順で処理が行われます：

- **自己注意機構** <br>
入力データの各部分の重要度を計算し、重要な部分に焦点を当てます。これにより、入力データの相互関係を学習します。
- **フィードフォワードネットワーク（FNN）**<br>
自己注意機構の出力に対して、2層の全結合ネットワークを適用します。これにより、特徴表現がさらに強化されます。
- **正規化と残差接続**<br>
各サブレイヤーの出力を正規化し、入力に加算することで、学習を安定させます。

In [ ]:
# TransformerEncoderLayerクラスの定義
class TransformerEncoderLayer(nn.Module):
    def __init__(self, dim_hidden, num_heads, dim_feedforward):
        super().__init__()
        self.attention = SelfAttention(dim_hidden, num_heads)  # 自己注意機構
        self.fnn = FNN(dim_hidden, dim_feedforward)  # フィードフォワードネットワーク
        self.norm1 = nn.LayerNorm(dim_hidden)  # 最初のLayerNorm
        self.norm2 = nn.LayerNorm(dim_hidden)  # 二番目のLayerNorm

    def forward(self, x):
        # 自己注意機構の出力に残差接続を追加
        x = x + self.attention(self.norm1(x))
        # フィードフォワードネットワークの出力に残差接続を追加
        x = x + self.fnn(self.norm2(x))
        return x

### SelfAttention クラスの定義

SelfAttention クラスは、自己注意機構を定義します。

CNN（Convolutional Neural Network）では、固定のカーネルを画像全体に対して施します。CNNの畳み込み層では、カーネル（フィルタ）が画像の各位置に対して適用され、特徴マップが生成されます。これにより、局所的な特徴を抽出することができますが、画像全体の文脈を考慮することは困難です。

ViTは、特徴量のうち **識別に有効な部分だけを重み付け** する効果があります。これを実現するのが自己注意機構（Self-Attention Mechanism）です。自己注意機構では、クエリとキーの内積を計算し、ソフトマックス関数を用いて重み付けを行います。この重みをバリューに掛けることで、重要な特徴量に対して高い重みを与え、そうでない部分には低い重みを与えることができます。

自己注意機構では、以下の手順で処理が行われます：

#### **クエリ（Query）、キー（Key）、バリュー（Value）** の計算

ViTでは、画像を小さなパッチに分割し、それぞれのパッチを特徴ベクトルに変換します。この特徴ベクトルを使って、クエリ（Query）、キー（Key）、バリュー（Value）という3つの新しい行列を計算します。クエリは、そのパッチが他のパッチとどれだけ関連があるかを測るための行列です。キーは、他のパッチと関連性を評価するための行列です。バリューは、パッチが持つ情報そのものを表す行列です。これらの行列を計算することで、ViTは画像全体の文脈を捉えることができます。

数式では、入力パッチの特徴ベクトルを$\mathbf{X}$とし、それに重み行列$\mathbf{W_Q}$、$\mathbf{W_K}$、$\mathbf{W_V}$を掛けることで、クエリ$\mathbf{Q}$、キー$\mathbf{K}$、バリュー$\mathbf{V}$を得ます。具体的には次のようになります：

\begin{align*}
\mathbf{Q} &= \mathbf{X} \mathbf{W_Q}, \\
\mathbf{K} &= \mathbf{X} \mathbf{W_K}, \\
\mathbf{V} &= \mathbf{X} \mathbf{W_V}
\end{align*}

ここで、$\mathbf{W_Q}$、$\mathbf{W_K}$、$\mathbf{W_V}$は学習可能なパラメータであり、訓練データを通じて最適化されます。これにより、各パッチが他のパッチとどのように関連しているかを学習することができます。結果として、画像全体の意味や文脈を理解するための重要な情報が得られます。

#### **注意スコア** の計算

次に、クエリとキーを使って、各パッチの関連性を計算します。これを注意スコアと呼びます。注意スコアは、各パッチが他のパッチとどれだけ関連しているかを示す値です。具体的には、クエリ$\mathbf{Q}$とキー$\mathbf{K}$の内積を計算し、それをキーの次元数$\sqrt{D}$で割って正規化します。この結果にソフトマックス関数を適用して、注意スコアを得ます。ソフトマックス関数を使うことで、注意スコアが確率分布となり、全てのパッチのスコアの合計が1になります。

数式では、注意スコアは次のように計算されます：

\begin{align*}
\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) &= \text{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{D}}\right)\mathbf{V}
\end{align*}

ここで、$\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V})$は注意機構の出力を表し、$\mathbf{Q}\mathbf{K}^T$はクエリとキーの内積を取った行列です。この内積は、各パッチが他のパッチにどれだけ注意を払うべきかを示します。次に、これを$\sqrt{D}$で割ることで、数値のスケールを調整します。

#### ソフトマックスおよびバリューの計算
ソフトマックス関数を適用することで、各パッチの重要度が確率として表現されます。最後に、この注意スコアをバリュー $\mathbf{V}$ に掛けることで、各パッチの情報を重み付けして集約します。これにより、ViTは画像全体の情報を効果的に集約し、より良い特徴量を得ることができます。

In [ ]:
# SelfAttentionクラスの定義
class SelfAttention(nn.Module):
    def __init__(self, dim_hidden, num_heads, qkv_bias=False):
        super().__init__()
        assert dim_hidden % num_heads == 0  # 隠れ次元がヘッドの数で割り切れるか確認
        self.num_heads = num_heads  # 注意機構のヘッドの数
        dim_head = dim_hidden // num_heads  # 各ヘッドの次元数
        self.scale = dim_head ** -0.5  # スケーリング係数

        # 入力をQ、K、Vに投影する線形層
        self.proj_in = nn.Linear(dim_hidden, dim_hidden * 3, bias=qkv_bias)
        self.proj_out = nn.Linear(dim_hidden, dim_hidden)  # 出力の線形層

    def forward(self, x):
        bs, n, _ = x.shape  # バッチサイズとシーケンス長
        # Q、K、Vに分割して計算
        qkv = self.proj_in(x).view(bs, n, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)  # クエリ、キー、バリューに分割

        # 注意スコアを計算
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)  # ソフトマックスで正規化

        # コンテキストベクトルを計算
        x = (attn @ v).transpose(1, 2).reshape(bs, n, -1)
        self.attn_weights = attn  # Attention weightsを保存
        return self.proj_out(x)

### FNNクラスの定義

FNNクラスは、フィードフォワードネットワーク（FNN）を定義します。FNNは、自己注意機構の出力に対して、さらに特徴を抽出するために使用されます。具体的には、2層の全結合ネットワークを使用します。

FNNでは、以下の手順で処理が行われます：

- 線形変換と活性化関数：入力データに対して線形変換を行い、その後GELU活性化関数を適用します。これにより、非線形性が導入されます。
- もう一つの線形変換：活性化関数の出力に対して、もう一度線形変換を行います。

In [ ]:
# FNNクラスの定義
class FNN(nn.Module):
    def __init__(self, dim_hidden, dim_feedforward):
        super().__init__()
        self.linear1 = nn.Linear(dim_hidden, dim_feedforward)  # 最初の線形変換
        self.linear2 = nn.Linear(dim_feedforward, dim_hidden)  # 次の線形変換
        self.activation = nn.GELU()  # 活性化関数

    def forward(self, x):
        # 線形変換 -> 活性化関数 -> 線形変換の順に適用
        return self.linear2(self.activation(self.linear1(x)))

#### **課題２**

- Query, Key, Value のうち、のちの処理に使うために出力される特徴量はどれですか？
 - その出力のために、上記以外の２つはどのような役割を果たしますか？
- 畳み込み層とは違って、ここでは特徴量の重み付けが行われます。そのメリットは何ですか？

---

### 学習

CNN が ViT に置き換わったものの、それに対する学習処理は CNN の場合とまったく同じです。

In [ ]:
# ViT クラスのインスタンスを net として得る
net = ViT()
print(net)

# GPUが利用可能な場合はGPUにモデルを移動させる
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

initial_lr = 0.01 # 初期学習率

optimizer = optim.SGD(net.parameters(), lr=initial_lr) # Adam オプティマイザ
criterion = nn.CrossEntropyLoss() # クロスエントロピー損失関数

データも準備します。

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 学習用データセットをロードし、検証用データセットに分割
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_size = int(0.8 * len(trainset))
validation_size = len(trainset) - train_size
train_dataset, validation_dataset = random_split(trainset, [train_size, validation_size])

# ミニバッチ (mini-batch) サイズを100とし、学習用データローダと検証用データローダを定義
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
validationloader = torch.utils.data.DataLoader(validation_dataset, batch_size=100, shuffle=False)

# テスト用データセットをロード
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

ここから学習ループに入ります。

ViT は CNN よりも処理が重いため、エポック数は少なめにしています。他は CNN の課題とほぼ同一です。



In [ ]:
max_epoch = 15     # 最大エポック数
patience = 5       # 改善が見られないエポック数の許容回数
trigger_times = 0  # 改善が見られないエポック数のカウンター

best_val_loss = float("inf")  # 初期値として無限大を設定
train_losses = []  # 学習データセットの損失を保存するリスト
val_losses = []  # 検証データセットの損失を保存するリスト

# エポック数のループ
for epoch in range(max_epoch):
    running_loss = 0.0

    # 学習データセットからミニバッチを得るたびに…
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)  # 入力データをGPUに送る
        labels = labels.to(device)  # ラベルをGPUに送る

        optimizer.zero_grad()  # 勾配の初期化

        outputs = net(inputs)  # ネットワークに入力データを渡して出力を取得
        loss = criterion(outputs, labels)  # 損失を計算
        loss.backward()  # 逆伝播を行い、勾配を計算
        optimizer.step()  # パラメータを更新

        running_loss += loss.item()  # ミニバッチの損失を累積

    # エポックごとの訓練データセットに対する平均損失を計算
    train_loss = running_loss / len(trainloader)
    train_losses.append(train_loss)

    # 検証データセットに対する損失を計算
    val_loss = 0.0
    net.eval()  # モデルを評価モードに切り替える
    with torch.no_grad():

        # 検証データセットからミニバッチを得るたびに…
        for data in validationloader:
            images, labels = data
            images = images.to(device)  # 入力データをGPUに送る
            labels = labels.to(device)  # ラベルをGPUに送る

            outputs = net(images)  # ネットワークに入力データを渡して出力を取得
            loss = criterion(outputs, labels)  # 損失を計算
            val_loss += loss.item()  # 損失を累積

    # エポックごとの検証データセットに対する平均損失を計算
    val_loss = val_loss / len(validationloader)
    val_losses.append(val_loss)

    # 損失の値をグラフで表示
    plt.clf()  # 前のグラフをクリア
    plt.plot(train_losses, label='Training loss')  # 訓練データセットの損失
    plt.plot(val_losses, label='Validation loss')  # 検証データセットの損失をプロット
    plt.xlabel('Epoch')  # x軸のラベルを設定
    plt.ylabel('Loss')  # y軸のラベルを設定
    plt.xlim(left=0)  # x軸の表示範囲を設定
    plt.ylim(bottom=0)  # y軸の表示範囲を設定
    plt.legend()  # 凡例を表示
    plt.show()  # グラフを表示

    # エポックごとの損失を出力
    print("Epoch:", epoch + 1)
    print("Train loss     : ", train_loss)
    print("Validation loss: ", val_loss)

    # 検証データセットに対する損失が改善しない場合の処理
    if val_loss < best_val_loss:
        best_val_loss = val_loss  # 最良の検証データセット損失を更新
        trigger_times = 0  # 改善が見られないエポック数をリセット
    else:
        trigger_times += 1  # 改善が見られないエポック数をカウントアップ
        if trigger_times >= patience:  # 許容回数を超えた場合の処理
            print(f"{epoch + 1} エポックで早期終了")  # 早期終了のメッセージを出力
            break  # 学習を終了

print('学習終了')

### テスト

精度を求める処理もまったく同じです。

CIFAR-10 は小規模なデータのため、残念ながら CNN を超える精度は出ないかもしれません。

In [ ]:
correct = 0  # 正解数のカウンターを初期化
total = 0  # 全体の画像数のカウンターを初期化

# 訓練モードでの計算を停止（推論モードに切り替える）
with torch.no_grad():
    # テストデータセットに対してループ
    for data in testloader:
        images, labels = data  # ミニバッチごとの画像とラベルを取得
        images = images.to(device)  # GPUに画像を送る
        labels = labels.to(device)  # GPUにラベルを送る

        outputs = net(images)  # ネットワークに画像を入力し、出力を取得
        _, predicted = torch.max(outputs.data, 1)  # 出力の中で最大の値を持つクラスを予測として取得
        total += labels.size(0)  # ミニバッチ内の画像数を全体の画像数に加算
        correct += (predicted == labels).sum().item()  # 予測が正しい場合、正解数をカウントアップ

print('テスト画像における精度 %d %%' % (100 * correct / total))

### クラスベクトル

クラスベクトルは、**クラストークン** がTransformerエンコーダを通過した後の最終的な表現を指します。言い換えれば、クラストークンがエンコーダを通じて得た情報を含むベクトルです。このクラスベクトルは、画像全体の特徴を捉えており、最終的な分類層に入力され、各クラスへの所属確率を出力します。まずそれ自体を表で確認しましょう。

In [ ]:
import pandas as pd

# CIFAR-10のクラス名
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

# 学習済みモデルのクラスベクトルを取得
class_vectors = net.linear.weight.data.cpu().numpy()

# クラス番号、クラス名、クラスベクトルをデータフレームに格納
df = pd.DataFrame(class_vectors, columns=[f'Feature_{i+1}' for i in range(class_vectors.shape[1])])
df.insert(0, 'Class Name', classes)
df.insert(0, 'Class Number', range(10))

# データフレームを表示
# import ace_tools as tools; tools.display_dataframe_to_user(name="Class Vectors", dataframe=df)
df

### クラスベクトルによる可視化

ここではクラスベクトルを用いて、入力画像のどの部分がモデルの予測に重要だったか（注意されていたか）をヒートマップで示します。セグメンテーションのように事前に注目箇所を与えて学習するわけではありませんが、Transformerの自己注意機構により、各パッチ間の関係性を学習し、自然に注目箇所が形成されます。各パッチが予測にどの程度寄与しているかを示す注意スコアを計算し、そのスコアを基にヒートマップを生成します。

ここで、ViTにおける自己注意機構をおさらいします：

**クエリ（Query）、キー（Key）、バリュー（Value）の計算**

\begin{align*}
\mathbf{Q} &= \mathbf{X} \mathbf{W_Q}, \
\mathbf{K} &= \mathbf{X} \mathbf{W_K}, \
\mathbf{V} &= \mathbf{X} \mathbf{W_V}
\end{align*}

ここで、$\mathbf{W_Q}$、$\mathbf{W_K}$、$\mathbf{W_V}$
$X$は入力パッチの特徴ベクトル、$\mathbf{W_Q}$、$\mathbf{W_K}$、$\mathbf{W_V}$はそれぞれの重み行列です。

**注意スコアの計算**

\begin{align*}
\text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) &= \text{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{D}}\right)\mathbf{V}
\end{align*}
$D$はキーの次元数です。

**クラスベクトルと注意スコアを用いたヒートマップの生成**

クラスベクトルは、各クラスに対応する特徴ベクトルであり、最終的なクラス分類に使用されます。注意スコアとバリュー行列を用いて、画像内の各パッチの重要性を示すヒートマップを生成することができます。具体的には、注意スコア$\alpha_i$と対応するバリューベクトル$\mathbf{V}_i$を掛け合わせて、各パッチの重要度を計算します。

数式では、ヒートマップは次のように計算されます：

\begin{align*}
\text{Heatmap} &= \sum_{i} \alpha_i \mathbf{V}_i
\end{align*}

ここで、$\alpha_i$は注意スコアであり、$\mathbf{V}_i$はバリュー行列$\mathbf{V}$の$i$列目のベクトルです。ヒートマップは、モデルが注目している各パッチの重要性を示します。このヒートマップを視覚化することで、モデルがどの部分に注目しているかを理解することができます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch
from PIL import Image

# テストデータの画像を表示し、正解と推定値を表示する関数
def visualize_test_predictions(start_idx, end_idx):
    # テストデータローダーを作成
    testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

    # 指定したインデックス範囲の画像とラベルを取得
    images, labels = zip(*[(data[0], data[1]) for i, data in enumerate(testloader) if start_idx <= i < end_idx])

    # モデルの予測結果を格納するリストを初期化
    predicted_labels = []

    # 推論モードで計算を停止
    with torch.no_grad():
        # 画像ごとに予測を行う
        for idx, image in enumerate(images):
            image_tensor = image.clone().detach().to(device)  # NumPy配列をテンソルに変換してGPUに送る
            outputs = net(image_tensor)  # ネットワークに画像を入力し、出力を得る
            _, predicted = torch.max(outputs, 1)  # 出力の中で最大の値を持つクラスを予測として取得
            predicted_labels.append(predicted.item())  # 予測結果をリストに追加

            # ヒートマップの計算
            class_vector = net.linear.weight[predicted].cpu().detach().numpy()
            attn = net.layers[0].attention.attn_weights.cpu().detach().numpy()  # attention weights

            # 各ヘッドの注意重みを平均化
            avg_attn = np.mean(attn[0,...], axis=0)

            # クラスベクトルを除いたパッチ数を計算
            num_patches = avg_attn.shape[-1] - 1  # クラスベクトルを除く
            patch_size = int(np.sqrt(num_patches))

            if patch_size * patch_size != num_patches:
                raise ValueError(f"Attention weights shape {num_patches} is not a perfect square")

            # クラスベクトルを除いてリシェイプ
            heatmap = avg_attn[1:, 1:]  # (64, 64) の形状
            heatmap = np.mean(heatmap, axis=0).reshape(patch_size, patch_size)  # (8, 8) の形状に平均化してリシェイプ
            heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())  # 正規化

            # ヒートマップを元の画像サイズに補間
            heatmap_resized = np.array(Image.fromarray(np.uint8(255 * heatmap)).resize((32, 32)))

            # オリジナル画像とヒートマップの重畳表示
            image_np = image.cpu().detach().squeeze().permute(1, 2, 0).numpy()
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(image_np)
            ax[0].set_title(f'Original Image\nClass: {classes[labels[idx]]}')
            ax[0].axis('off')

            ax[1].imshow(image_np)
            ax[1].imshow(heatmap_resized, alpha=0.7)
            ax[1].set_title(f'Heatmap Overlay\nPredicted: {classes[predicted]}')
            ax[1].axis('off')

            plt.show()

# 例: インデックス範囲 0 から 5 の画像を表示
visualize_test_predictions(0, 5)

こちらも、あまり期待する結果ではなかったかもしれません。CIFAR-10 は ViT の真価を発揮するにはデータが小さく、Google Colab での学習もあまり多く行うことはできません。十分な計算資源がある場合は、ImageNet のような大規模データセットを用いて事前学習してから、この学習を行うといった案も考えられます。

#### **課題３**

- ここで可視化されているヒートマップのうち，値の高いところ（赤や黄色）は何を意味しますか？
 - これを求めるために行われている計算を説明してください
 - **Grad-CAM** （グラッドカム）も今回と似たヒートマップを出力する手法で、CNN でも使用できます。[ふんわり理解するGrad-CAM -- Zenn](https://zenn.dev/iq108uni/articles/7269a1b72f42be)<br>
 CNN に対して Grad-CAM で行っていることと、ViT に対してここで行っていることは何が違いますか？